Web Scraping data

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time

# Select season year
num_of_seasons = 2
season_begin_year = 2022
all_matches = []

for seasonNum in range(int(num_of_seasons)):
	season_begin_year = season_begin_year + seasonNum
	season_end_year = season_begin_year + 1

	standings_url = f"https://fbref.com/en/comps/9/{season_begin_year}-{season_end_year}/{season_begin_year}-{season_end_year}-Premier-League-Stats"

	season_year = standings_url.split("/")[-2]

	data = requests.get(standings_url)
	soup = BeautifulSoup(data.text)

	standings_table = soup.select('table.stats_table')[0]

	all_links = [l.get('href') for l in standings_table.find_all('a')] 

	# Get information link for each team
	team_links = [l for l in all_links if '/squads/' in l]
	team_urls = [f'https://fbref.com{l}' for l in team_links]

	for team_url in team_urls:
		team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
		data = requests.get(team_url)

		matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
		
		soup = BeautifulSoup(data.text)
		links = soup.find_all('a')
		links = [l.get("href") for l in links]
		shooting_links = [l for l in links if l and 'all_comps/shooting/' in l]

		shooting_data = requests.get(f"https://fbref.com{shooting_links[0]}")
		shooting = pd.read_html(StringIO(shooting_data.text), match="Shooting")[0]
		shooting.columns = shooting.columns.droplevel()
		
		try:
			shooting_selected = shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]]
			team_data = matches.merge(shooting_selected, on="Date")
		except ValueError:
			continue
		
		team_epl_data = team_data[team_data["Comp"] == "Premier League"].copy()
		team_epl_data["Season"] = season_year
		team_epl_data["Team"] = team_name
		all_matches.append(team_epl_data)
		print(f'{len(all_matches)}: {team_name}')
		time.sleep(10)

	print(f'Season {season_begin_year}-{season_end_year} completed')
	match_df = pd.concat(all_matches)

1: Manchester City
2: Arsenal
3: Manchester United
4: Newcastle United
5: Liverpool
6: Brighton and Hove Albion
7: Aston Villa
8: Tottenham Hotspur
9: Brentford
10: Fulham
11: Crystal Palace
12: Chelsea
13: Wolverhampton Wanderers
14: West Ham United
15: Bournemouth
16: Nottingham Forest
17: Everton
18: Leicester City
19: Leeds United
20: Southampton
Season 2022-2023 completed
21: Manchester City
22: Arsenal
23: Liverpool
24: Aston Villa
25: Tottenham Hotspur
26: Chelsea
27: Newcastle United
28: Manchester United
29: West Ham United
30: Crystal Palace
31: Brighton and Hove Albion
32: Bournemouth
33: Fulham
34: Wolverhampton Wanderers
35: Everton
36: Brentford
37: Nottingham Forest
38: Luton Town
39: Burnley
40: Sheffield United
Season 2023-2024 completed


In [15]:
# Check data information
match_df.info()
match_df

<class 'pandas.core.frame.DataFrame'>
Index: 1520 entries, 1 to 40
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           1520 non-null   object 
 1   Time           1520 non-null   object 
 2   Comp           1520 non-null   object 
 3   Round          1520 non-null   object 
 4   Day            1520 non-null   object 
 5   Venue          1520 non-null   object 
 6   Result         1520 non-null   object 
 7   GF             1520 non-null   object 
 8   GA             1520 non-null   object 
 9   Opponent       1520 non-null   object 
 10  xG             1520 non-null   float64
 11  xGA            1520 non-null   float64
 12  Poss           1520 non-null   float64
 13  Attendance     1518 non-null   float64
 14  Captain        1520 non-null   object 
 15  Formation      1520 non-null   object 
 16  Opp Formation  1520 non-null   object 
 17  Referee        1520 non-null   object 
 18  Match Report   

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,Match Report,NaN,13.0,1.0,18.7,1.0,1,1,2022-2023,Manchester City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,Match Report,NaN,19.0,7.0,17.5,0.0,0,0,2022-2023,Manchester City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,Match Report,NaN,21.0,10.0,16.2,1.0,0,0,2022-2023,Manchester City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,Match Report,NaN,18.0,5.0,14.1,0.0,0,0,2022-2023,Manchester City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,Match Report,NaN,17.0,9.0,14.8,0.0,0,0,2022-2023,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,...,Match Report,NaN,10.0,4.0,17.8,1.0,0,0,2023-2024,Sheffield United
37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,...,Match Report,NaN,15.0,4.0,13.5,0.0,0,0,2023-2024,Sheffield United
38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,...,Match Report,NaN,16.0,4.0,18.0,0.0,1,1,2023-2024,Sheffield United
39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,Everton,...,Match Report,NaN,13.0,1.0,21.0,0.0,0,0,2023-2024,Sheffield United


In [16]:
# Export cleaned data set
match_df.to_csv(f'eql_matchData_22_24.csv', encoding="utf-8-sig")

Data Cleaning

In [3]:
import os
import pandas as pd

curr_dir = os.getcwd()
file_name = "eql_matchData_22_24.csv"
file_path = os.path.join(curr_dir, file_name)

# Load Data file
epl_df = pd.read_csv(file_path, index_col=0)

# Drop unnessary columns
epl_df.drop(['Formation', 'Opp Formation', 'Match Report', 'Notes'], axis=1, inplace=True)

# Convert date string to datetime
epl_df['Date'] = pd.to_datetime(epl_df['Date'])

epl_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Captain,Referee,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,İlkay Gündoğan,Michael Oliver,13.0,1.0,18.7,1.0,1,1,2022-2023,Manchester City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,İlkay Gündoğan,David Coote,19.0,7.0,17.5,0.0,0,0,2022-2023,Manchester City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,İlkay Gündoğan,Jarred Gillett,21.0,10.0,16.2,1.0,0,0,2022-2023,Manchester City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,Kevin De Bruyne,Darren England,18.0,5.0,14.1,0.0,0,0,2022-2023,Manchester City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,İlkay Gündoğan,Paul Tierney,17.0,9.0,14.8,0.0,0,0,2022-2023,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,...,Oliver Arblaster,Michael Salisbury,10.0,4.0,17.8,1.0,0,0,2023-2024,Sheffield United
37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,...,Oliver Arblaster,Tony Harrington,15.0,4.0,13.5,0.0,0,0,2023-2024,Sheffield United
38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,...,Jack Robinson,Chris Kavanagh,16.0,4.0,18.0,0.0,1,1,2023-2024,Sheffield United
39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,Everton,...,Jack Robinson,Stuart Attwell,13.0,1.0,21.0,0.0,0,0,2023-2024,Sheffield United


In [2]:
epl_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1520 entries, 1 to 40
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1520 non-null   datetime64[ns]
 1   Time        1520 non-null   object        
 2   Comp        1520 non-null   object        
 3   Round       1520 non-null   object        
 4   Day         1520 non-null   object        
 5   Venue       1520 non-null   object        
 6   Result      1520 non-null   object        
 7   GF          1520 non-null   int64         
 8   GA          1520 non-null   int64         
 9   Opponent    1520 non-null   object        
 10  xG          1520 non-null   float64       
 11  xGA         1520 non-null   float64       
 12  Poss        1520 non-null   float64       
 13  Attendance  1518 non-null   float64       
 14  Captain     1520 non-null   object        
 15  Referee     1520 non-null   object        
 16  Sh          1520 non-null   flo

Relevant data for the match prediction
- Venue (Home, Away)
- Opponent (Team name)
- Time (Hour only)
- Day of week
- Result (Win, Draw, or Lose)

In [4]:
# Create numeric codes for prediction
epl_df["Venue_code"] = epl_df["Venue"].astype("category").cat.codes
epl_df["Opp_code"] = epl_df["Opponent"].astype("category").cat.codes

# Modify time variable (Extract hour from the time variable)
epl_df["hour"] = epl_df["Time"].str.replace(":.+", "", regex=True).astype("int")

# Create a numeric value for each day of week
epl_df["day_code"] = epl_df["Date"].dt.dayofweek

# Convert match results to numeric variable named 'result_num'
epl_df["Target"] = epl_df["Result"].apply(lambda x: 2 if x == 'W' else (1 if x == 'D' else 0)).astype("int")

# Convert season to numeric
epl_df["Season_num"] = epl_df["Season"].apply(lambda x: 1 if x == '2022-2023' else 2)
epl_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,PK,PKatt,Season,Team,Venue_code,Opp_code,hour,day_code,Target,Season_num
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,1,1,2022-2023,Manchester City,0,21,16,6,2,1
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,0,0,2022-2023,Manchester City,1,2,15,5,2,1
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,0,0,2022-2023,Manchester City,0,16,16,6,1,1
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,0,0,2022-2023,Manchester City,1,7,15,5,2,1
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,0,0,2022-2023,Manchester City,1,17,19,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,...,0,0,2023-2024,Sheffield United,0,15,20,2,0,2
37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,...,0,0,2023-2024,Sheffield United,0,16,15,5,0,2
38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,...,1,1,2023-2024,Sheffield United,1,17,15,5,0,2
39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,Everton,...,0,0,2023-2024,Sheffield United,0,8,15,5,0,2


Machine Learning Prediction
 - Random Forest
	- Used for a Non-Linear model
	- Series of decision trees

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# Initialize randomforest class
rf = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=42)

train_data = epl_df[epl_df["Season_num"] == 1]
test_data = epl_df[epl_df["Season_num"] == 2]

predictors = ["Venue_code", "Opp_code", "hour", "day_code"]

rf.fit(train_data[predictors], train_data["Target"])

RandomForestClassifier(min_samples_split=10, random_state=42)

In [23]:
preds = rf.predict(test_data[predictors])
print(preds)
print(len(preds))

[1 0 0 2 0 2 0 0 1 0 2 1 2 2 0 0 2 1 2 0 2 0 2 2 0 2 2 0 0 2 0 2 0 2 2 1 0
 2 2 1 2 2 0 2 0 2 1 2 0 1 0 2 0 0 0 0 2 2 1 0 2 2 0 0 0 2 0 0 2 2 2 2 2 0
 0 2 1 2 0 0 2 2 0 0 2 2 0 1 0 2 0 0 2 0 0 0 0 2 0 1 0 0 1 0 0 2 0 2 2 1 2
 2 0 2 0 2 0 0 2 0 0 2 2 2 0 2 2 0 0 0 0 2 0 1 2 1 0 2 0 2 0 2 2 2 0 1 0 2
 2 0 0 1 0 0 0 0 2 0 0 0 2 1 2 2 0 0 2 1 2 1 0 0 0 0 0 1 2 2 0 1 2 0 1 0 0
 2 0 1 1 2 2 2 0 2 0 0 2 0 0 0 0 2 0 0 0 2 2 2 2 0 2 0 2 1 0 1 0 1 0 0 1 0
 0 1 2 0 1 2 0 0 2 2 2 2 1 2 2 0 0 2 0 0 2 2 2 0 2 0 0 0 2 0 2 0 2 1 2 2 0
 2 1 2 0 1 0 0 2 0 2 0 1 1 2 1 2 0 1 2 0 0 2 2 0 2 2 0 2 2 2 0 0 2 0 2 0 2
 2 2 2 1 1 0 0 0 0 2 2 0 0 0 2 1 0 2 1 2 0 2 2 2 2 0 0 2 0 2 0 0 1 2 2 0 0
 0 2 0 2 0 2 0 2 0 1 2 1 2 0 2 0 2 0 2 0 1 0 2 2 2 0 2 1 1 0 2 0 2 1 1 0 2
 1 2 0 0 2 0 0 0 0 2 2 0 2 1 0 0 0 2 0 2 2 2 1 0 0 1 0 2 1 0 1 0 2 0 0 1 0
 2 0 1 0 0 0 0 0 0 2 2 2 0 2 1 0 0 0 2 2 1 0 1 1 0 1 0 1 0 2 2 2 2 0 0 0 2
 2 0 1 2 0 0 0 2 0 0 1 1 2 1 0 0 2 0 2 2 0 0 0 0 2 0 2 2 0 1 2 0 0 2 0 2 2
 0 1 0 0 1 0 0 2 2 2 1 0 

Analyze prediction accuracy

In [18]:
# Get prediction accuracy
acc = accuracy_score(test_data["Target"], preds)
print(f'Accuracy: {acc:.2f}')

Accuracy: 0.43


In [19]:
pred_result = pd.DataFrame(dict(actual=test_data["Target"], prediction=preds))

pred_table = pd.crosstab(index=pred_result["actual"], columns=pred_result["prediction"], margins=True)
pred_table

prediction,0,1,2,All
actual,,,,
0,167,37,94,298
1,70,29,65,164
2,124,46,128,298
All,361,112,287,760


In [21]:
# Calculate Precision
precision_macro = precision_score(test_data["Target"], preds, average='macro')

print(f'Macro Precision: {precision_macro:.2f}')

Macro Precision: 0.39


Rolling Average for each team's performance

In [22]:
# Group by team
team_match = epl_df.groupby("Team")
team = team_match.get_group("Manchester City")
team

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,PK,PKatt,Season,Team,Venue_code,Opp_code,hour,day_code,Target,Season_num
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,1,1,2022-2023,Manchester City,0,21,16,6,2,1
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,0,0,2022-2023,Manchester City,1,2,15,5,2,1
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,0,0,2022-2023,Manchester City,0,16,16,6,1,1
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,0,0,2022-2023,Manchester City,1,7,15,5,2,1
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,0,0,2022-2023,Manchester City,1,17,19,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,2024-04-28,16:30,Premier League,Matchweek 35,Sun,Away,W,2,0,Nott'ham Forest,...,0,0,2023-2024,Manchester City,0,17,16,6,2,2
52,2024-05-04,17:30,Premier League,Matchweek 36,Sat,Home,W,5,1,Wolves,...,2,2,2023-2024,Manchester City,1,22,17,5,2,2
53,2024-05-11,12:30,Premier League,Matchweek 37,Sat,Away,W,4,0,Fulham,...,1,1,2023-2024,Manchester City,0,9,12,5,2,2
54,2024-05-14,20:00,Premier League,Matchweek 34,Tue,Away,W,2,0,Tottenham,...,1,1,2023-2024,Manchester City,0,20,20,1,2,2


In [30]:
# Rolling average function
def rolling_average(team, vars, vars_rolling):
	# Sort data by date
	team = team.sort_values("Date")

	# Perform rolling average - Calculate average based on the data in previous 3 games
	rolling_stats = round(team[vars].rolling(3, closed='left').mean(), 4)
	team[vars_rolling] = rolling_stats

	# NA for first 3 games
	team[vars_rolling].fillna("NA")

	return team


cols = ["GF", "GA", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]
new_cols = [f"{c}_rolling" for c in cols]

# Rolling average data for Machester City
rolling_average(team, cols, new_cols)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Target,Season_num,GF_rolling,GA_rolling,Sh_rolling,SoT_rolling,Dist_rolling,FK_rolling,PK_rolling,PKatt_rolling
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,2,1,3.0000,1.0000,17.6667,6.0000,17.4667,0.6667,0.3333,0.3333
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,2,1,3.6667,1.6667,19.3333,7.3333,15.9333,0.3333,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,2024-04-28,16:30,Premier League,Matchweek 35,Sun,Away,W,2,0,Nott'ham Forest,...,2,2,4.3333,1.0000,22.6667,8.6667,16.6667,0.3333,0.3333,0.3333
52,2024-05-04,17:30,Premier League,Matchweek 36,Sat,Home,W,5,1,Wolves,...,2,2,3.6667,0.3333,20.3333,7.6667,17.5333,0.6667,0.3333,0.3333
53,2024-05-11,12:30,Premier League,Matchweek 37,Sat,Away,W,4,0,Fulham,...,2,2,3.6667,0.3333,14.3333,7.0000,17.0000,0.6667,0.6667,0.6667
54,2024-05-14,20:00,Premier League,Matchweek 34,Tue,Away,W,2,0,Tottenham,...,2,2,3.6667,0.3333,14.6667,7.6667,17.2000,0.3333,1.0000,1.0000


In [44]:
# Add rolling average to the epl_df
def apply_rolling_average(group, cols, new_cols):
    return rolling_average(group, cols, new_cols)

match_data = epl_df.groupby("Team").apply(apply_rolling_average, cols=cols, new_cols=new_cols, include_groups=False)

# Set unique index number
match_data.index = range(match_data.shape[0])

match_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Target,Season_num,GF_rolling,GA_rolling,Sh_rolling,SoT_rolling,Dist_rolling,FK_rolling,PK_rolling,PKatt_rolling
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2,0,Crystal Palace,...,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,2,Leicester City,...,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3,0,Bournemouth,...,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,Fulham,...,2,1,3.0000,0.6667,14.3333,5.0000,14.1333,0.3333,0.0000,0.0000
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2,1,Aston Villa,...,2,1,3.0000,1.0000,18.3333,7.0000,14.4333,0.3333,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,2024-04-24,19:45,Premier League,Matchweek 29,Wed,Home,L,0,1,Bournemouth,...,0,2,1.0000,2.0000,9.6667,4.0000,20.5333,0.3333,0.3333,0.3333
1516,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Home,W,2,1,Luton Town,...,2,2,0.6667,1.6667,10.3333,3.3333,18.7667,0.0000,0.0000,0.0000
1517,2024-05-04,17:30,Premier League,Matchweek 36,Sat,Away,L,1,5,Manchester City,...,0,2,0.6667,1.3333,11.0000,4.0000,19.6667,0.0000,0.0000,0.0000
1518,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Home,L,1,3,Crystal Palace,...,0,2,1.0000,2.3333,10.0000,3.3333,15.9667,0.0000,0.0000,0.0000
